In [ ]:
#Reshaping and pivot tables******************8

In [4]:
import numpy as np
import pandas as pd
import pandas.util.testing as tm
tm.N = 3


def unpivot(frame):
    N, K = frame.shape
    data = {'value': frame.to_numpy().ravel('F'),
            'variable': np.asarray(frame.columns).repeat(N),
            'date': np.tile(np.asarray(frame.index), K)}
    return pd.DataFrame(data, columns=['date', 'variable', 'value'])


df = unpivot(tm.makeTimeDataFrame())


In [13]:
df.describe()

,value
count,12.000000
mean,0.221325
std,0.905546
min,-1.132317
25%,-0.634276
50%,0.450578
75%,0.967707
max,1.291056


In [6]:
#o select out everything for variable A we could do:

In [7]:
df[df['variable'] == 'A']

,date,variable,value
0,2000-01-03,A,1.291056
1,2000-01-04,A,0.192474
2,2000-01-05,A,1.153700


In [8]:
#But suppose we wish to do time series operations with the variables. 

In [9]:
df.pivot(index='date', columns='variable', values='value')

variable,A,B,C,D
date,,,,
2000-01-03,1.291056,-0.848981,-1.132317,0.384684
2000-01-04,0.192474,1.205096,-1.080590,0.905709
2000-01-05,1.153700,-0.562707,0.631308,0.516471


In [14]:
#input DataFrame has more than one column of values

In [15]:
df['value2'] = df['value'] * 2

In [16]:
df.head()

,date,variable,value,value2
0,2000-01-03,A,1.291056,2.582112
1,2000-01-04,A,0.192474,0.384948
2,2000-01-05,A,1.153700,2.307399
3,2000-01-03,B,-0.848981,-1.697962
4,2000-01-04,B,1.205096,2.410191


In [17]:
pivoted = df.pivot(index='date', columns='variable')

In [18]:
pivoted

value                                  value2            \
variable           A         B         C         D         A         B   
date                                                                     
2000-01-03  1.291056 -0.848981 -1.132317  0.384684  2.582112 -1.697962   
2000-01-04  0.192474  1.205096 -1.080590  0.905709  0.384948  2.410191   
2000-01-05  1.153700 -0.562707  0.631308  0.516471  2.307399 -1.125415   

                                
variable           C         D  
date                            
2000-01-03 -2.264634  0.769369  
2000-01-04 -2.161180  1.811418  
2000-01-05  1.262616  1.032943

In [19]:
pivoted['value2']

variable,A,B,C,D
date,,,,
2000-01-03,2.582112,-1.697962,-2.264634,0.769369
2000-01-04,0.384948,2.410191,-2.161180,1.811418
2000-01-05,2.307399,-1.125415,1.262616,1.032943


In [ ]:
###Reshaping by stacking and unstacking

In [29]:
#stack: “pivot” a level of the (possibly hierarchical) column labels, returning a DataFrame with an index with a new inner-most level of row labels.

In [30]:
#unstack: (inverse operation of stack) “pivot” a level of the (possibly hierarchical) row index to the column axis, producing a reshaped DataFrame with a new inner-most level of column labels.

In [20]:
tuples = list(zip(*[['bar', 'bar', 'baz', 'baz','foo', 'foo', 'qux', 'qux'],['one', 'two', 'one', 'two','one', 'two', 'one', 'two']]))

In [22]:
index = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])

In [23]:
df = pd.DataFrame(np.random.randn(8, 2), index=index, columns=['A', 'B'])

In [38]:
df.head()

A         B
first second                    
bar   one    -0.047432 -0.196817
      two     1.323534  0.483401
baz   one     0.711108  0.376672
      two     0.801471  0.195738
foo   one     0.431079 -0.376920

In [24]:
df2 = df[:4]

In [25]:
df2

A         B
first second                    
bar   one    -0.047432 -0.196817
      two     1.323534  0.483401
baz   one     0.711108  0.376672
      two     0.801471  0.195738

In [26]:
stacked = df2.stack()

In [27]:
stacked

first  second   
bar    one     A   -0.047432
               B   -0.196817
       two     A    1.323534
               B    0.483401
baz    one     A    0.711108
               B    0.376672
       two     A    0.801471
               B    0.195738
dtype: float64

In [31]:
stacked.unstack(0)

first          bar       baz
second                      
one    A -0.047432  0.711108
       B -0.196817  0.376672
two    A  1.323534  0.801471
       B  0.483401  0.195738

In [32]:
#Reshaping by Melt

In [33]:
#The top-level melt() function and the corresponding DataFrame.melt() are useful to massage a DataFrame into a format where one or more columns are identifier variables, while all other columns, considered measured variables, are “unpivoted” to the row axis, leaving just two non-identifier columns, “variable” and “value”. The names of those columns can be customized by supplying the var_name and value_name parameters.

In [34]:
cheese = pd.DataFrame({'first': ['John', 'Mary'],
                       'last': ['Doe', 'Bo'],
                     'height': [5.5, 6.0],
                     'weight': [130, 150]})

In [35]:
cheese

,first,last,height,weight
0,John,Doe,5.5,130
1,Mary,Bo,6.0,150


In [36]:
cheese.melt(id_vars=['first', 'last'])

,first,last,variable,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


In [37]:
cheese.melt(id_vars=['first', 'last'], var_name='quantity')

,first,last,quantity,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


In [40]:
import datetime

In [41]:
df = pd.DataFrame({'A': ['one', 'one', 'two', 'three'] * 6,
                  'B': ['A', 'B', 'C'] * 8,
                 'C': ['foo', 'foo', 'foo', 'bar', 'bar', 'bar'] * 4,
                 'D': np.random.randn(24),
                   'E': np.random.randn(24),
                  'F': [datetime.datetime(2013, i, 1) for i in range(1, 13)]
                   + [datetime.datetime(2013, i, 15) for i in range(1, 13)]})


In [42]:
df.head()

,A,B,C,D,E,F
0,one,A,foo,-0.477552,-1.556687,2013-01-01
1,one,B,foo,2.313058,-0.419664,2013-02-01
2,two,C,foo,0.630174,0.711270,2013-03-01
3,three,A,bar,-0.605493,-1.798174,2013-04-01
4,one,B,bar,0.406475,-0.217488,2013-05-01


In [44]:
pd.pivot_table(df, values='D', index=['A', 'B'], columns=['C'])

C             bar       foo
A     B                    
one   A  0.812372 -0.060548
      B -0.569615  0.930876
      C  0.354376 -0.999135
three A -0.642468       NaN
      B       NaN  0.152144
      C -0.427702       NaN
two   A       NaN -1.507431
      B -1.224241       NaN
      C       NaN  1.240603

In [45]:
#Adding margins

In [46]:
df.pivot_table(index=['A', 'B'], columns='C', margins=True, aggfunc=np.std)

D                             E                    
C             bar       foo       All       bar       foo       All
A     B                                                            
one   A  1.177665  0.589732  0.912262  0.678600  1.262682  0.915453
      B  1.380400  1.954700  1.630729  0.385742  0.160347  0.318972
      C  0.904094  0.478142  0.979456  0.099469  0.304086  0.526470
three A  0.052290       NaN  0.052290  1.355009       NaN  1.355009
      B       NaN  1.791080  1.791080       NaN  2.598546  2.598546
      C  1.729097       NaN  1.729097  1.513658       NaN  1.513658
two   A       NaN  1.113716  1.113716       NaN  0.743091  0.743091
      B  0.752124       NaN  0.752124  1.244538       NaN  1.244538
      C       NaN  0.863277  0.863277       NaN  0.954735  0.954735
All      1.091981  1.380948  1.198021  0.994054  1.107526  1.012809

In [ ]:
#https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html